# Example-01: Derivative

In [1]:
# Given an input function, its higher order (partial) derivatives with respect to one or sevaral tensor arguments can be computed using forward or reverse mode automatic differentiation
# Derivative orders can be different for each tensor argument
# Input function is expected to return a tensor or a (nested) list of tensors

# Derivatives are computed by nesting functorch jacobian functions
# For higher order derivatives, this results in growing redundant computations, forward mode is more efficient in this case

# If the input function returns a tensor, the output is referred as derivative table representation
# This representation can be evaluated near given evaluation point if the input function returns a scalar or a vector
# Table representation is a (nested) list of tensors, it can be used as a (redundant) function representation near given evaluation point
# Table structure for f(x), f(x, y) and f(x, y, z) is shown bellow (similar structure holds for a function with more aruments)

# f(x)
# t(f, x)
# [f, Dx f, Dxx f, ...]

# f(x, y)
# t(f, x, y)
# [
#     [    f,     Dy f,     Dyy f, ...],
#     [ Dx f,  Dx Dy f,  Dx Dyy f, ...],
#     [Dxx f, Dxx Dy f, Dxx Dyy f, ...],
#     ...
# ]

# f(x, y, z)
# t(f, x, y, z)
# [
#     [
#         [         f,          Dz f,          Dzz f, ...],
#         [      Dy f,       Dy Dz f,       Dy Dzz f, ...],
#         [     Dyy f,      Dyy Dz f,      Dyy Dzz f, ...],
#         ...
#     ],
#     [
#         [      Dx f,       Dx Dz f,       Dx Dzz f, ...],
#         [   Dx Dy f,    Dx Dy Dz f,    Dx Dy Dzz f, ...],
#         [  Dx Dyy f,   Dx Dyy Dz f,   Dx Dyy Dzz f, ...],
#         ...
#     ],
#     [
#         [    Dxx f,     Dxx Dz f,     Dxx Dzz f, ...],
#         [ Dxx Dy f,  Dxx Dy Dz f,  Dxx Dy Dzz f, ...],
#         [Dxx Dyy f, Dxx Dyy Dz f, Dxx Dyy Dzz f, ...],
#         ...
#     ],
#     ...
# ]

In [2]:
# Import

import torch
import functorch

import sys
sys.path.append('..')

import ndtorch.ndtorch as nd

torch.set_printoptions(precision=12, sci_mode=True)
print(torch.cuda.is_available())

True


In [3]:
# Set data type and device

dtype = torch.float64
device = torch.device('cpu')

In [4]:
# Basic derivative interface

# nd.derivative(
#     order:int,                            # derivative order
#     function:Callable,                    # input function
#     *args,                                # function(*args) = function(x:Tensor, ...)
#     intermediate:bool = True,             # flag to return intermediate derivatives
#     jacobian:Callable = functorch.jacfwd  # functorch.jacfwd or functorch.jacfrev
# )

# nd.derivative(
#     order:tuple[int, ...],                # derivative orders
#     function:Callable,                    # input function
#     *args,                                # function(*args) = function(x:Tensor, y:Tensor, z:Tensor, ...)
#     intermediate:bool = True,             # flag to return intermediate derivatives
#     jacobian:Callable = functorch.jacfwd  # functorch.jacfwd or functorch.jacfrev
# )

In [5]:
# Derivative

# Input:  scalar
# Output: scalar

# Set test function
# Note, the first function argument is a scalar tensor
# Input function can have other additional arguments
# Other arguments are not used in computation of derivatives

def fn(x, a, b, c, d, e, f):
    return a + b*x + c*x**2 + d*x**3 + e*x**4 + f*x**5

# Set derivative order

n = 5

# Set evaluation point

x = torch.tensor(0.0, dtype=dtype, device=device)

# Set fixed parameters

a, b, c, d, e, f = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0], dtype=dtype, device=device)

# Compute n'th derivative

value = nd.derivative(n, fn, x, a, b, c, d, e, f, intermediate=False, jacobian=functorch.jacfwd)
print(value.cpu().numpy().tolist())

# Compute all derivatives upto given order
# Note, function value itself is referred as zeros order derivative
# Since function returns a tensor, output is a list of tensors

values = nd.derivative(n, fn, x, a, b, c, d, e, f, intermediate=True, jacobian=functorch.jacfwd)
print(*[value.cpu().numpy().tolist() for value in values], sep=', ')

# Note, intermediate flag (default=True) can be used to return all derivatives
# For jacobian parameter, functorch.jacfwd or functorch.jacrev functions can be passed

# Evaluate derivative table representation for a given deviation from the evaluation point

dx = torch.tensor(1.0, dtype=dtype, device=device)
print(nd.evaluate(nd.derivative(n, fn, x, a, b, c, d, e, f) , [dx]).cpu().numpy().tolist())
print(fn(x + dx, a, b, c, d, e, f).cpu().numpy().tolist())

120.0
1.0, 1.0, 2.0, 6.0, 24.0, 120.0
6.0
6.0


In [6]:
# Derivative

# Input:  vector
# Output: scalar

# Set test function
# Note, the first function argument is a vector tensor
# Input function can have other additional arguments
# Other arguments are not used in computation of derivatives

def fn(x, a, b, c):
    x1, x2 = x
    return a + b*(x1 - 1)**2 + c*(x2 + 1)**2

# Set derivative order

n = 2

# Set evaluation point

x = torch.tensor([0.0, 0.0], dtype=dtype, device=device)

# Set fixed parameters

a, b, c = torch.tensor([1.0, 1.0, 1.0], dtype=dtype, device=device)

# Compute only n'th derivative
# Note, for given input & output the result is hessian

value = nd.derivative(n, fn, x, a, b, c, intermediate=False, jacobian=functorch.jacfwd)
print(value.cpu().numpy().tolist())

# Compute all derivatives upto given order
# Note, fuction value itself is referred as zeros order derivative
# Output is a list of tensors (value, jacobian, hessian, ...)

values = nd.derivative(n, fn, x, a, b, c, intermediate=True, jacobian=functorch.jacfwd)
print(*[value.cpu().numpy().tolist() for value in values], sep=', ')

# Compute jacobian and hessian with functorch

print(fn(x, a, b, c).cpu().numpy().tolist(), 
      functorch.jacfwd(lambda x: fn(x, a, b, c))(x).cpu().numpy().tolist(), 
      functorch.hessian(lambda x: fn(x, a, b, c))(x).cpu().numpy().tolist(), 
      sep=', ')

# Evaluate derivative table representation for a given deviation from the evaluation point

dx = torch.tensor([+1.0, -1.0], dtype=dtype, device=device)
print(nd.evaluate(values, [dx]).cpu().numpy())
print(fn(x + dx, a, b, c).cpu().numpy())

# Evaluate can be mapped over a set of deviation values

print(functorch.vmap(lambda x: nd.evaluate(values, [x]))(torch.stack(5*[dx])).cpu().numpy().tolist())

# Derivative can be mapped over a set of evaluation points
# Note, the inputt function is expeted to return a tensor

print(functorch.vmap(lambda x: nd.derivative(1, fn, x, a, b, c, intermediate=False))(torch.stack(5*[x])).cpu().numpy().tolist())

[[2.0, 0.0], [0.0, 2.0]]
3.0, [-2.0, 2.0], [[2.0, 0.0], [0.0, 2.0]]
3.0, [-2.0, 2.0], [[2.0, 0.0], [0.0, 2.0]]
1.0
1.0
[1.0, 1.0, 1.0, 1.0, 1.0]
[[-2.0, 2.0], [-2.0, 2.0], [-2.0, 2.0], [-2.0, 2.0], [-2.0, 2.0]]


In [7]:
# Derivative

# Input:  vector
# Output: vector

# Set test function
# Note, the first function argument is a vector tensor
# Input function can have other additional arguments
# Other arguments (if any) are not used in computation of derivatives

def fn(x):
    x1, x2 = x
    X1 = 1.0*x1 + 2.0*x2
    X2 = 3.0*x1 + 4.0*x2
    X3 = 5.0*x1 + 6.0*x2
    return torch.stack([X1, X2, X3])

# Set derivative order

n = 1

# Set evaluation point

x = torch.tensor([0.0, 0.0], dtype=dtype, device=device)

# Compute derivatives

values = nd.derivative(n, fn, x)
print(*[value.cpu().numpy().tolist() for value in values], sep=', ')
print()

# Evaluate derivative table representation for a given deviation from the evaluation point

dx = torch.tensor([+1, -1], dtype=dtype, device=device)
print(nd.evaluate(values, [dx]).cpu().numpy().tolist())
print(fn(x + dx).cpu().numpy().tolist())

[0.0, 0.0, 0.0], [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]

[-1.0, -1.0, -1.0]
[-1.0, -1.0, -1.0]


In [8]:
# Derivative

# Input:  tensor
# Output: tensor

# Set test function

def fn(x):
    return 1 + x + x**2 + x**3

# Set derivative order

n = 3

# Set evaluation point

x = torch.zeros((1, 2, 3), dtype=dtype, device=device)

# Compute derivatives
# Note, output is a list of tensors

values = nd.derivative(n, fn, x)
print(*[list(value.shape) for value in values], sep='\n')

# Evaluate derivative table representation for a given deviation from the evaluation point
# Note, evaluate function works with scalar or vector tensor input
# One should compute derivatives of a wrapped function and reshape the result of evaluate

# Set wrapped function

def gn(x, shape):
    return fn(x.reshape(shape)).flatten()

print(fn(x).cpu().numpy().tolist())
print(gn(x.flatten(), x.shape).reshape(x.shape).cpu().numpy().tolist())

# Compute derivatives

values = nd.derivative(n, gn, x.flatten(), x.shape)

# Set deviation value

dx = torch.ones_like(x)

# Evaluate

print(nd.evaluate(values, [dx.flatten()]).reshape(x.shape).cpu().numpy().tolist())
print(gn((x + dx).flatten(), x.shape).reshape(x.shape).cpu().numpy().tolist())
print(fn(x + dx).cpu().numpy().tolist())

[1, 2, 3]
[1, 2, 3, 1, 2, 3]
[1, 2, 3, 1, 2, 3, 1, 2, 3]
[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]
[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]]
[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]]
[[[4.0, 4.0, 4.0], [4.0, 4.0, 4.0]]]
[[[4.0, 4.0, 4.0], [4.0, 4.0, 4.0]]]
[[[4.0, 4.0, 4.0], [4.0, 4.0, 4.0]]]


In [9]:
# Derivative

# Input:  vector
# Output: nested list of tensors

# Set test function

def fn(x):
    x1, x2, x3, x4, x5, x6 = x
    X1 = 1.0*x1 + 2.0*x2 + 3.0*x3
    X2 = 4.0*x4 + 5.0*x5 + 6.0*x6
    return [torch.stack([X1]), [torch.stack([X2])]]

# Set derivative order

n = 1

# Set evaluation point

x = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=dtype, device=device)

# Compute derivatives

values = nd.derivative(n, fn, x, intermediate=False)

In [10]:
# Derivative

# Input:  vector, vector, vector
# Output: vector

# Set test function

def fn(x, y, z):
    x1, x2 = x
    y1, y2 = y
    z1, z2 = z
    return torch.stack([(x1 + x2)*(y1 + y2)*(z1 + z2)])

# Set derivative orders for x, y and z

nx, ny, nz = 1, 1, 1

# Set evaluation point
# Note, evaluation point is a list of tensors

x = torch.tensor([0.0, 0.0], dtype=dtype, device=device)
y = torch.tensor([0.0, 0.0], dtype=dtype, device=device)
z = torch.tensor([0.0, 0.0], dtype=dtype, device=device)

# Compute n'th derivativ

value = nd.derivative((nx, ny, nz), fn, x, y, z, intermediate=False)
print(value.cpu().numpy().tolist())

# Compute all derivatives upto given order

values = nd.derivative((nx, ny, nz), fn, x, y, z, intermediate=True)

# Evaluate derivative table representation for a given deviation from the evaluation point

dx = torch.tensor([1.0, 1.0], dtype=dtype, device=device)
dy = torch.tensor([1.0, 1.0], dtype=dtype, device=device)
dz = torch.tensor([1.0, 1.0], dtype=dtype, device=device)
print(nd.evaluate(values, [dx, dy, dz]).cpu().numpy().tolist())
print(fn(x + dx, y + dy, z + dz).cpu().numpy().tolist())

# Note, if the input function has vector arguments and returns a tensor, it can be repsented with series

for key, value in nd.series(tuple(map(len, (x, y, z))), (nx, ny, nz), values).items():
    print(f'{key}: {value.cpu().numpy().tolist()}')

[[[[1.0, 1.0], [1.0, 1.0]], [[1.0, 1.0], [1.0, 1.0]]]]
[8.0]
[8.0]
(0, 0, 0, 0, 0, 0): [0.0]
(0, 0, 0, 0, 1, 0): [0.0]
(0, 0, 0, 0, 0, 1): [0.0]
(0, 0, 1, 0, 0, 0): [0.0]
(0, 0, 0, 1, 0, 0): [0.0]
(0, 0, 1, 0, 1, 0): [0.0]
(0, 0, 1, 0, 0, 1): [0.0]
(0, 0, 0, 1, 1, 0): [0.0]
(0, 0, 0, 1, 0, 1): [0.0]
(1, 0, 0, 0, 0, 0): [0.0]
(0, 1, 0, 0, 0, 0): [0.0]
(1, 0, 0, 0, 1, 0): [0.0]
(1, 0, 0, 0, 0, 1): [0.0]
(0, 1, 0, 0, 1, 0): [0.0]
(0, 1, 0, 0, 0, 1): [0.0]
(1, 0, 1, 0, 0, 0): [0.0]
(1, 0, 0, 1, 0, 0): [0.0]
(0, 1, 1, 0, 0, 0): [0.0]
(0, 1, 0, 1, 0, 0): [0.0]
(1, 0, 1, 0, 1, 0): [1.0]
(1, 0, 1, 0, 0, 1): [1.0]
(1, 0, 0, 1, 1, 0): [1.0]
(1, 0, 0, 1, 0, 1): [1.0]
(0, 1, 1, 0, 1, 0): [1.0]
(0, 1, 1, 0, 0, 1): [1.0]
(0, 1, 0, 1, 1, 0): [1.0]
(0, 1, 0, 1, 0, 1): [1.0]


In [11]:
# Redundancy free computation

# Set test function

def fn(x):
    x1, x2 = x
    return torch.stack([1.0*x1 + 2.0*x2 + 3.0*x1**2 + 4.0*x1*x2 + 5.0*x2**2])

# Set derivative order

n = 2

# Set evaluation point

x = torch.tensor([0.0, 0.0], dtype=dtype, device=device)

# Compute n'th derivative

value = nd.derivative(n, fn, x, intermediate=False)
print(value.cpu().numpy().tolist())

# Since derivatives are computed by nesting of jacobian function, redundant computations appear starting from the second order
# Redundant computations can be avoided if all input arguments are scalar tensors

def gn(x1, x2):
    return fn(torch.stack([x1, x2]))

print(nd.derivative((2, 0), gn, *x, intermediate=False).cpu().numpy().tolist())
print(nd.derivative((1, 1), gn, *x, intermediate=False).cpu().numpy().tolist())
print(nd.derivative((0, 2), gn, *x, intermediate=False).cpu().numpy().tolist())

[[[6.0, 4.0], [4.0, 10.0]]]
[6.0]
[4.0]
[10.0]


# Example-02: Derivative table representation

In [1]:
# Input function f: R^n x R^m x ... -> R^n is referred as a mapping
# The first function argument is state, other arguments (used in computation of derivatives) and knobs
# State and all knobs are vector-like tensors
# Note, functions of this form can be used to model tranformations throught accelerator magnets

# In this case, derivatives can be used to generate a (parametric) model of the input function
# Function model can be represented as a derivative table or coefficients of monomials (series representation)

# In this example, table representation is used to model transformation throught a sextupole accelerator magnet
# Table is computed with respect to state variables (phase space variables) and knobs (magnet strength and length)

In [2]:
# Import

import numpy
import torch
import functorch

import sys
sys.path.append('..')

import ndtorch.ndtorch as nd

torch.set_printoptions(precision=12, sci_mode=True)
print(torch.cuda.is_available())

True


In [3]:
# Set data type and device

dtype = torch.float64
device = torch.device('cpu')

In [4]:
# Mapping (sextupole accelerator magnet transformation)
# Given initial state, magnet strength and length, state is propagated using explicit symplectic integration
# Number of integration steps is set by count parameter, integration step length is length/count

def mapping(x, k, l, count=100):
    (qx, px, qy, py), (k, ), (l, ) = x, k, l/(2.0*count)
    for _ in range(count):
        qx, qy = qx + l*px, qy + l*py
        px, py = px - 2.0*l*k*(qx**2 - qy**2), py + 2.0*l*k*qx*qy
        qx, qy = qx + l*px, qy + l*py
    return torch.stack([qx, px, qy, py])

In [5]:
# Table representation (state)

# Set evaluation point & parameters

x = torch.tensor([0.0, 0.0, 0.0, 0.0], dtype=dtype, device=device)
k = torch.tensor([10.0], dtype=dtype, device=device)
l = torch.tensor([0.1], dtype=dtype, device=device)

# Compute derivatives (table representation)
# Since derivatives are computed only with respect to the state, output table is a list of tensors

t = nd.derivative(6, mapping, x, k, l)

print(*[element.shape for element in t], sep='\n')

torch.Size([4])
torch.Size([4, 4])
torch.Size([4, 4, 4])
torch.Size([4, 4, 4, 4])
torch.Size([4, 4, 4, 4, 4])
torch.Size([4, 4, 4, 4, 4, 4])
torch.Size([4, 4, 4, 4, 4, 4, 4])


In [6]:
# Compare table and exact mapping near the evaluation point (change order to observe convergence)
# Note, table transformation is not symplectic

dx = torch.tensor([0.0, 0.001, 0.0001, 0.0], dtype=dtype, device=device)

print(nd.evaluate(t, [dx]).cpu().tolist())
print(mapping(x + dx, k, l).cpu().tolist())

[0.00010000041666220641, 0.001000006666736112, 0.00010000016667544442, 5.000018331681037e-09]
[0.00010000041666220632, 0.001000006666736112, 0.00010000016667544451, 5.000018331681034e-09]


In [7]:
# Each bottom element (tensor) in the (flattend) derivative table is assosiated with a signature
# Signature is a tuple of derivative orders

print(nd.signature(t))

[(0,), (1,), (2,), (3,), (4,), (5,), (6,)]


In [8]:
# For a given signature, corresponding element can be extracted or changed with get/set functions

print(nd.get(t, (1, )).cpu().numpy())

[[1.  0.1 0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0.1]
 [0.  0.  0.  1. ]]


In [9]:
# Each bottom element is related to monomials
# For given order, monomial indices with repetitions can be computed
# These repetitions account for evaluation of the same partial derivatives with diffenent orders, e.g. df/dxdy vs df/dydx

print(nd.index(4, 2).cpu().numpy())

[[2 0 0 0]
 [1 1 0 0]
 [1 0 1 0]
 [1 0 0 1]
 [1 1 0 0]
 [0 2 0 0]
 [0 1 1 0]
 [0 1 0 1]
 [1 0 1 0]
 [0 1 1 0]
 [0 0 2 0]
 [0 0 1 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 1]
 [0 0 0 2]]


In [10]:
# Explicit evaluation

print(nd.evaluate(t, [dx]).cpu().numpy())
print((t[0] + t[1] @ dx + 1/2 * t[2] @ dx @ dx + 1/2 * 1/3 * t[3] @ dx @ dx @ dx + 1/2 * 1/3 * 1/4 * t[4] @ dx @ dx @ dx @ dx + 1/2 * 1/3 * 1/4 * 1/5 * t[5] @ dx @ dx @ dx @ dx @ dx + 1/2 * 1/3 * 1/4 * 1/5 * 1/6 * t[6] @ dx @ dx @ dx @ dx @ dx @ dx).cpu().numpy())
print((t[0] + (t[1] + 1/2 * (t[2] + 1/3 * (t[3] + 1/4 * (t[4] + 1/5 * (t[5] + 1/6 * t[6] @ dx) @ dx) @ dx) @ dx) @ dx) @ dx).cpu().numpy())

[1.00000417e-04 1.00000667e-03 1.00000167e-04 5.00001833e-09]
[1.00000417e-04 1.00000667e-03 1.00000167e-04 5.00001833e-09]
[1.00000417e-04 1.00000667e-03 1.00000167e-04 5.00001833e-09]


In [11]:
# Series representation can be generated from a given table
# This representation stores monomial powers and corresponding coefficients

s = nd.series((4, ), (6, ), t)
print(torch.stack([s[(1, 0, 0, 0)], s[(0, 1, 0, 0)], s[(0, 0, 1, 0)], s[(0, 0, 0, 1)]]).cpu().numpy())

[[1.  0.  0.  0. ]
 [0.1 1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.1 1. ]]


In [12]:
# Evaluate series

print(nd.evaluate(t, [dx]).cpu().numpy())
print(nd.evaluate(s, [dx]).cpu().numpy())

[1.00000417e-04 1.00000667e-03 1.00000167e-04 5.00001833e-09]
[1.00000417e-04 1.00000667e-03 1.00000167e-04 5.00001833e-09]


In [13]:
# Table representation (state & knobs)

# Set evaluation point

x = torch.tensor([0.0, 0.0, 0.0, 0.0], dtype=dtype, device=device)
k = torch.tensor([10.0], dtype=dtype, device=device)
l = torch.tensor([0.1], dtype=dtype, device=device)

# Compute derivatives (table representation)
# Since derivatives are computed with respect to state and knobs, output table is a nested list of tensors

t = nd.derivative((6, 1, 1), mapping, x, k, l)

In [14]:
# In this case, bottom table element signature is a tuple with several integers

print(nd.get(t, (1, 0, 0)).cpu().numpy())

[[1.  0.1 0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0.1]
 [0.  0.  0.  1. ]]


In [15]:
# Compare table and exact mapping near evaluation point (change order to observe convergence)
# Note, table transofrmation is not symplectic

dx = torch.tensor([0.0, 0.001, 0.0001, 0.0], dtype=dtype, device=device)
dk = torch.tensor([0.1], dtype=dtype, device=device)
dl = torch.tensor([0.001], dtype=dtype, device=device)

print(nd.evaluate(t, [dx, 0.0*dk, 0.0*dl]).cpu().tolist())
print(nd.evaluate(t, [dx, 1.0*dk, 1.0*dl]).cpu().tolist())
print(mapping(x + dx, k + dk, l + dl).cpu().tolist())

[0.00010000041666220641, 0.001000006666736112, 0.00010000016667544442, 5.000018331681037e-09]
[0.00010100042756197639, 0.0010000067334053492, 0.0001000001733924745, 5.151019293264671e-09]
[0.00010100042756163604, 0.0010000067323920011, 0.0001000001734431457, 5.151524301990432e-09]


In [16]:
# Each bottom element (tensor) in the (flattend) derivative table is assosiated with a signature
# Signature is a tuple of derivative orders

print(*[index for index in nd.signature(t)], sep='\n')

(0, 0, 0)
(0, 0, 1)
(0, 1, 0)
(0, 1, 1)
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)
(2, 0, 0)
(2, 0, 1)
(2, 1, 0)
(2, 1, 1)
(3, 0, 0)
(3, 0, 1)
(3, 1, 0)
(3, 1, 1)
(4, 0, 0)
(4, 0, 1)
(4, 1, 0)
(4, 1, 1)
(5, 0, 0)
(5, 0, 1)
(5, 1, 0)
(5, 1, 1)
(6, 0, 0)
(6, 0, 1)
(6, 1, 0)
(6, 1, 1)


In [17]:
# Compute series

s = nd.series((4, 1, 1), (6, 1, 1), t)

# Keys are generalized monomials

print(s[(1, 1, 1, 1, 1, 1)].cpu().numpy())
print()

# Evaluate series

print(nd.evaluate(t, [dx, dk, dl]).cpu().numpy())
print(nd.evaluate(s, [dx, dk, dl]).cpu().numpy())
print()

[1.42877133e-06 9.99755017e-05 0.00000000e+00 0.00000000e+00]

[1.01000428e-04 1.00000673e-03 1.00000173e-04 5.15101929e-09]
[1.01000428e-04 1.00000673e-03 1.00000173e-04 5.15101929e-09]

